### In this notebook we perform Centralized learning, almost like 02.Individual_Training. 

In centralized learning there is an entity that has access to data from all base stations.
Here, there is no option to filter out any base station.
In this setting we also measure the energy consumption using the Carbontracker tool.

In [1]:
import sys
import os

from pathlib import Path

parent = Path(os.path.abspath("")).resolve().parents[0]
if parent not in sys.path:
    sys.path.insert(0, str(parent))

In [2]:
import random

import numpy as np
import torch

from argparse import Namespace

In [3]:
from ml.utils.data_utils import read_data, generate_time_lags, time_to_feature, handle_nans, to_Xy, \
    to_torch_dataset, to_timeseries_rep, assign_statistics, \
    to_train_val, scale_features, get_data_by_area, remove_identifiers, get_exogenous_data_by_area, handle_outliers

In [4]:
from ml.utils.train_utils import train, test

In [5]:
from ml.models.mlp import MLP
from ml.models.rnn import RNN
from ml.models.lstm import LSTM
from ml.models.gru import GRU
from ml.models.cnn import CNN
from ml.models.rnn_autoencoder import DualAttentionAutoEncoder

In [6]:
args = Namespace(
    data_path='../dataset/turbine_prediction_selected_features_400turbines.csv', # dataset
    data_path_test=None , # test dataset
    test_size=0.2, # validation size
    targets=['power_output'], # the target columns
    num_lags= 24, # the number of past observations to feed as input

    
    filter_bs=None, # whether to use a single bs for training. It will be changed dynamically
    identifier='GSRN', # the column name that identifies a bs

    nan_constant=0, # the constant to transform nan values
    x_scaler='minmax', # x_scaler
    y_scaler='minmax', # y_scaler
    outlier_detection=None, # whether to perform flooring and capping

    
    criterion='mse', # optimization criterion, mse or l1
    epochs=50, # the number of maximum epochs
    lr=0.001, # learning rate
    optimizer='adam', # the optimizer, it can be sgd or adam
    batch_size=1024, # the batch size to use
    early_stopping=True, # whether to use early stopping
    patience=10, # patience value for the early stopping parameter (if specified)
    max_grad_norm=0.0, # whether to clip grad norm
    reg1=0.0, # l1 regularization
    reg2=0.0, # l2 regularization
    
    plot_history=True, # plot loss history

    cuda=True, # whether to use gpu
    
    seed=0, # reproducibility

    assign_stats=None, # whether to use statistics as exogenous data, ["mean", "median", "std", "variance", "kurtosis", "skew"]
    use_time_features=False # whether to use datetime features
)

In [7]:
print(f"Script arguments: {args}\n")

Script arguments: Namespace(data_path='../dataset/turbine_prediction_selected_features_400turbines.csv', data_path_test=None, test_size=0.2, targets=['power_output'], num_lags=24, filter_bs=None, identifier='GSRN', nan_constant=0, x_scaler='minmax', y_scaler='minmax', outlier_detection=None, criterion='mse', epochs=50, lr=0.001, optimizer='adam', batch_size=1024, early_stopping=True, patience=10, max_grad_norm=0.0, reg1=0.0, reg2=0.0, plot_history=True, cuda=True, seed=0, assign_stats=None, use_time_features=False)



In [8]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("the device currently used is", device)

the device currently used is mps


In [9]:
# Outlier detection specification
if args.outlier_detection is not None:
    outlier_columns = ['rb_down', 'rb_up', 'down', 'up']
    outlier_kwargs = {"ElBorn": (10, 90), "LesCorts": (10, 90), "PobleSec": (5, 95)}
    args.outlier_columns = outlier_columns
    args.outlier_kwargs = outlier_kwargs

In [10]:
def seed_all():
    # ensure reproducibility
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [11]:
seed_all()

### By setting filter_bs to None, the preprocessing pipeline returns data from all three base stations

In [12]:
clients_to_skip = [
    "570714700000004469",
    "570714700000004568",
    "570714700000004704",
    "570714700000004964",
    "570714700000005060",
    "570714700000005619",
    "570714700000005626",
    "570714700000006029",
    "570714700000006296",
    "570714700000006890",
    "570714700000006982",
    "570714700000007194",
    "570714700000008252",
    "570714700000008627",
    "570715000000083464",
]

# Remove skipped clients (for DataFrame structure)
def filter_df_by_gsrn(df, clients_to_skip):
    if df is None:
        return None
    return df[~df["GSRN"].isin(clients_to_skip)].copy()


In [13]:
def make_preprocessing(filter_bs=None):
    """Preprocess a given .csv"""
    # read data
    df = read_data(args.data_path, filter_data=filter_bs)

    # filter out clients to skip in cluster 4
    df = filter_df_by_gsrn(df, clients_to_skip)
    print(f"Filtered out {len(clients_to_skip)} clients. Remaining rows: {len(df)}")
    
    # handle nans
    df = handle_nans(train_data=df, constant=args.nan_constant,
                     identifier=args.identifier)
    # split to train/validation
    train_data, val_data = to_train_val(df)
    
    # handle outliers (if specified)
    if args.outlier_detection is not None:
        train_data = handle_outliers(df=train_data, columns=args.outlier_columns,
                                     identifier=args.identifier, kwargs=args.outlier_kwargs)
    
    # get X and y
    X_train, X_val, y_train, y_val = to_Xy(train_data=train_data, val_data=val_data,
                                          targets=args.targets)
    
    # drop cluster columns from X
    if "cluster" in X_train.columns:
        X_train = X_train.drop(columns=["cluster"])
    if "cluster" in X_val.columns:
        X_val = X_val.drop(columns=["cluster"])
    
    # scale X
    X_train, X_val, x_scaler = scale_features(train_data=X_train, val_data=X_val,
                                             scaler=args.x_scaler, identifier=args.identifier)
    # scale y
    y_train, y_val, y_scaler = scale_features(train_data=y_train, val_data=y_val,
                                             scaler=args.y_scaler, identifier=args.identifier)
    
    # generate time lags
    X_train = generate_time_lags(X_train, args.num_lags)
    X_val = generate_time_lags(X_val, args.num_lags)
    y_train = generate_time_lags(y_train, args.num_lags, is_y=True)
    y_val = generate_time_lags(y_val, args.num_lags, is_y=True)
    
    # get datetime features as exogenous data
    date_time_df_train = time_to_feature(
        X_train, args.use_time_features, identifier=args.identifier
    )
    date_time_df_val = time_to_feature(
        X_val, args.use_time_features, identifier=args.identifier
    )
    
    # get statistics as exogenous data
    stats_df_train = assign_statistics(X_train, args.assign_stats, args.num_lags,
                                       targets=args.targets, identifier=args.identifier)
    stats_df_val = assign_statistics(X_val, args.assign_stats, args.num_lags, 
                                       targets=args.targets, identifier=args.identifier)
    
    # concat the exogenous features (if any) to a single dataframe
    if date_time_df_train is not None or stats_df_train is not None:
        exogenous_data_train = pd.concat([date_time_df_train, stats_df_train], axis=1)
        # remove duplicate columns (if any)
        exogenous_data_train = exogenous_data_train.loc[:, ~exogenous_data_train.columns.duplicated()].copy()
        assert len(exogenous_data_train) == len(X_train) == len(y_train)
    else:
        exogenous_data_train = None
    if date_time_df_val is not None or stats_df_val is not None:
        exogenous_data_val = pd.concat([date_time_df_val, stats_df_val], axis=1)
        exogenous_data_val = exogenous_data_val.loc[:, ~exogenous_data_val.columns.duplicated()].copy()
        assert len(exogenous_data_val) == len(X_val) == len(y_val)
    else:
        exogenous_data_val = None
        
    return X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler

In [14]:
X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler = make_preprocessing()

Filtered out 15 clients. Remaining rows: 3504000


/Users/libowen/Library/Python/3.10/lib/python/site-packages/sklearn/impute/_base.py:590: FutureWarning: Currently, when `keep_empty_feature=False` and `strategy="constant"`, empty features are not dropped. This behaviour will change in version 1.8. Set `keep_empty_feature=True` to preserve this behaviour.
  warnings.warn(
INFO logger 2025-11-01 02:09:41,473 | data_utils.py:393 | Observations info in 571313174116253585
INFO logger 2025-11-01 02:09:41,473 | data_utils.py:394 | 	Total number of samples:  8760
INFO logger 2025-11-01 02:09:41,474 | data_utils.py:395 | 	Number of samples for training: 7008
INFO logger 2025-11-01 02:09:41,474 | data_utils.py:396 | 	Number of samples for validation:  1752
INFO logger 2025-11-01 02:09:41,478 | data_utils.py:393 | Observations info in 570714700000106958
INFO logger 2025-11-01 02:09:41,479 | data_utils.py:394 | 	Total number of samples:  8760
INFO logger 2025-11-01 02:09:41,479 | data_utils.py:395 | 	Number of samples for training: 7008
INFO logg

In [15]:
X_train.head()

,Capacity_kw_lag-24,power_output_lag-24,age_lag-24,wind_dir_cos_lag-24,wind_dir_sin_lag-24,hour_cos_lag-24,hour_sin_lag-24,temperature_c_lag-24,wind_speed_lag-24,Capacity_kw_lag-23,...,Capacity_kw_lag-1,power_output_lag-1,age_lag-1,wind_dir_cos_lag-1,wind_dir_sin_lag-1,hour_cos_lag-1,hour_sin_lag-1,temperature_c_lag-1,wind_speed_lag-1,GSRN
time,,,,,,,,,,,,,,,,,,,,,
2019-01-02 00:00:00,0.230769,0.163281,0.052632,0.3875,0.01280,1.00000,0.50000,0.318182,0.525490,0.230769,...,0.230769,0.221352,0.052632,0.98065,0.36220,0.98295,0.37060,0.240642,0.839216,571313174116253585
2019-01-02 01:00:00,0.230769,0.174871,0.052632,0.3622,0.01935,0.98295,0.62940,0.318182,0.564706,0.230769,...,0.230769,0.221129,0.052632,0.98905,0.39605,1.00000,0.50000,0.221925,0.823529,571313174116253585
2019-01-02 02:00:00,0.230769,0.209959,0.052632,0.3372,0.02725,0.93300,0.75000,0.318182,0.564706,0.230769,...,0.230769,0.221195,0.052632,0.99385,0.42180,0.98295,0.62940,0.211230,0.784314,571313174116253585
2019-01-02 03:00:00,0.230769,0.208609,0.052632,0.3290,0.03015,0.85355,0.85355,0.318182,0.572549,0.230769,...,0.230769,0.221206,0.052632,0.99725,0.44775,0.93300,0.75000,0.200535,0.737255,571313174116253585
2019-01-02 04:00:00,0.230769,0.213543,0.052632,0.3538,0.02185,0.75000,0.93300,0.328877,0.623529,0.230769,...,0.230769,0.221263,0.052632,0.99880,0.46510,0.85355,0.85355,0.195187,0.705882,571313174116253585


In [16]:
y_train.head()

,power_output,GSRN
time,,
2019-01-02 00:00:00,0.221129,571313174116253585
2019-01-02 01:00:00,0.221195,571313174116253585
2019-01-02 02:00:00,0.221206,571313174116253585
2019-01-02 03:00:00,0.221263,571313174116253585
2019-01-02 04:00:00,0.221120,571313174116253585


In [17]:
x_scaler, y_scaler

(MinMaxScaler(), MinMaxScaler())

In [18]:
def make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler):
    """Make data ready to be fed into ml algorithms"""
    # if there are more than one specified areas, get the data per area
    if X_train[args.identifier].nunique() != 1:
        area_X_train, area_X_val, area_y_train, area_y_val = get_data_by_area(X_train, X_val,
                                                                              y_train, y_val, 
                                                                              identifier=args.identifier)
    else:
        area_X_train, area_X_val, area_y_train, area_y_val = None, None, None, None

    # Get the exogenous data per area.
    if exogenous_data_train is not None:
        exogenous_data_train, exogenous_data_val = get_exogenous_data_by_area(exogenous_data_train,
                                                                              exogenous_data_val)
    # transform to np
    if area_X_train is not None:
        for area in area_X_train:
            tmp_X_train, tmp_y_train, tmp_X_val, tmp_y_val = remove_identifiers(
                area_X_train[area], area_y_train[area], area_X_val[area], area_y_val[area])
            tmp_X_train, tmp_y_train = tmp_X_train.to_numpy(), tmp_y_train.to_numpy()
            tmp_X_val, tmp_y_val = tmp_X_val.to_numpy(), tmp_y_val.to_numpy()
            area_X_train[area] = tmp_X_train
            area_X_val[area] = tmp_X_val
            area_y_train[area] = tmp_y_train
            area_y_val[area] = tmp_y_val
    
    if exogenous_data_train is not None:
        for area in exogenous_data_train:
            exogenous_data_train[area] = exogenous_data_train[area].to_numpy()
            exogenous_data_val[area] = exogenous_data_val[area].to_numpy()
    
    # remove identifiers from features, targets
    X_train, y_train, X_val, y_val = remove_identifiers(X_train, y_train, X_val, y_val)
    assert len(X_train.columns) == len(X_val.columns)
    
    num_features = len(X_train.columns) // args.num_lags
    
    # to timeseries representation
    X_train = to_timeseries_rep(X_train.to_numpy(), num_lags=args.num_lags,
                                            num_features=num_features)
    X_val = to_timeseries_rep(X_val.to_numpy(), num_lags=args.num_lags,
                                          num_features=num_features)
    
    if area_X_train is not None:
        area_X_train = to_timeseries_rep(area_X_train, num_lags=args.num_lags,
                                                     num_features=num_features)
        area_X_val = to_timeseries_rep(area_X_val, num_lags=args.num_lags,
                                                   num_features=num_features)
    
    # transform targets to numpy
    y_train, y_val = y_train.to_numpy(), y_val.to_numpy()
    
    # centralized (all) learning specific
    if not args.filter_bs and exogenous_data_train is not None:
        exogenous_data_train_combined, exogenous_data_val_combined = [], []
        for area in exogenous_data_train:
            exogenous_data_train_combined.extend(exogenous_data_train[area])
            exogenous_data_val_combined.extend(exogenous_data_val[area])
        exogenous_data_train_combined = np.stack(exogenous_data_train_combined)
        exogenous_data_val_combined = np.stack(exogenous_data_val_combined)
        exogenous_data_train["all"] = exogenous_data_train_combined
        exogenous_data_val["all"] = exogenous_data_val_combined
    return X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val

In [19]:
X_train, X_val, y_train, y_val, area_X_train, area_X_val, area_y_train, area_y_val, exogenous_data_train, exogenous_data_val = make_postprocessing(X_train, X_val, y_train, y_val, exogenous_data_train, exogenous_data_val, x_scaler, y_scaler)

In [20]:
area_X_train.keys()

dict_keys([np.int64(571313174116253585), np.int64(570714700000106958), np.int64(570714700000004575), np.int64(570714700000004605), np.int64(570714700000004568), np.int64(570714700000005213), np.int64(570714700000004810), np.int64(570714700000004964), np.int64(570714700000004513), np.int64(570715000000107078), np.int64(570714700000106699), np.int64(570714700000106682), np.int64(570714700000005060), np.int64(570714700000004476), np.int64(570714700000004469), np.int64(570714700000004735), np.int64(570714700000004742), np.int64(570714700000004759), np.int64(570714700000003585), np.int64(570714700000011825), np.int64(570714700000011832), np.int64(570714700000005053), np.int64(570714700000004520), np.int64(570714700000010750), np.int64(570714700000010767), np.int64(570714700000004728), np.int64(570714700000004704), np.int64(570714700000012396), np.int64(570714700000005039), np.int64(570714700000009648), np.int64(570714700000009655), np.int64(570714700000009945), np.int64(570714700000009938),

In [21]:
area_X_val.keys()

dict_keys([np.int64(571313174116253585), np.int64(570714700000106958), np.int64(570714700000004575), np.int64(570714700000004605), np.int64(570714700000004568), np.int64(570714700000005213), np.int64(570714700000004810), np.int64(570714700000004964), np.int64(570714700000004513), np.int64(570715000000107078), np.int64(570714700000106699), np.int64(570714700000106682), np.int64(570714700000005060), np.int64(570714700000004476), np.int64(570714700000004469), np.int64(570714700000004735), np.int64(570714700000004742), np.int64(570714700000004759), np.int64(570714700000003585), np.int64(570714700000011825), np.int64(570714700000011832), np.int64(570714700000005053), np.int64(570714700000004520), np.int64(570714700000010750), np.int64(570714700000010767), np.int64(570714700000004728), np.int64(570714700000004704), np.int64(570714700000012396), np.int64(570714700000005039), np.int64(570714700000009648), np.int64(570714700000009655), np.int64(570714700000009945), np.int64(570714700000009938),

In [22]:
X_train[:2]

array([[[[2.30769232e-01],
         [1.63281038e-01],
         [5.26315756e-02],
         [3.87499988e-01],
         [1.28000081e-02],
         [1.00000000e+00],
         [5.00000000e-01],
         [3.18181813e-01],
         [5.25490224e-01]],

        [[2.30769232e-01],
         [1.74870551e-01],
         [5.26315756e-02],
         [3.62200022e-01],
         [1.93499923e-02],
         [9.82949972e-01],
         [6.29400015e-01],
         [3.18181813e-01],
         [5.64705908e-01]],

        [[2.30769232e-01],
         [2.09958583e-01],
         [5.26315756e-02],
         [3.37199986e-01],
         [2.72499919e-02],
         [9.32999969e-01],
         [7.50000000e-01],
         [3.18181813e-01],
         [5.64705908e-01]],

        [[2.30769232e-01],
         [2.08609119e-01],
         [5.26315756e-02],
         [3.28999996e-01],
         [3.01499963e-02],
         [8.53549957e-01],
         [8.53549957e-01],
         [3.18181813e-01],
         [5.72549045e-01]],

        [[2.30769232

In [23]:
y_train[:2]

array([[0.22112884],
       [0.22119494]], dtype=float32)

In [24]:
len(X_train), len(X_val)

(2793600, 691200)

In [25]:
def get_input_dims(X_train, exogenous_data_train):
    if args.model_name == "mlp":
        input_dim = X_train.shape[1] * X_train.shape[2]
    else:
        input_dim = X_train.shape[2]
    
    if exogenous_data_train is not None:
        if len(exogenous_data_train) == 1:
            cid = next(iter(exogenous_data_train.keys()))
            exogenous_dim = exogenous_data_train[cid].shape[1]
        else:
            exogenous_dim = exogenous_data_train["all"].shape[1]
    else:
        exogenous_dim = 0
    
    return input_dim, exogenous_dim

In [26]:
def get_model(model: str,
              input_dim: int,
              out_dim: int,
              lags: int = 10,
              exogenous_dim: int = 0,
              seed=0):
    if model == "mlp":
        model = MLP(input_dim=input_dim, layer_units=[256, 128, 64], num_outputs=out_dim)
    elif model == "rnn":
        model = RNN(input_dim=input_dim, rnn_hidden_size=128, num_rnn_layers=1, rnn_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "lstm":
        model = LSTM(input_dim=input_dim, lstm_hidden_size=128, num_lstm_layers=1, lstm_dropout=0.0,
                     layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "gru":
        model = GRU(input_dim=input_dim, gru_hidden_size=128, num_gru_layers=1, gru_dropout=0.0,
                    layer_units=[128], num_outputs=out_dim, matrix_rep=True, exogenous_dim=exogenous_dim)
    elif model == "cnn":
        model = CNN(num_features=input_dim, lags=lags, exogenous_dim=exogenous_dim, out_dim=out_dim)
    elif model == "da_encoder_decoder":
        model = DualAttentionAutoEncoder(input_dim=input_dim, architecture="lstm", matrix_rep=True)
    else:
        raise NotImplementedError("Specified model is not implemented. Plese define your own model or choose one from ['mlp', 'rnn', 'lstm', 'gru', 'cnn', 'da_encoder_decoder']")
    return model

In [27]:
# define the model
args.model_name = "lstm"

input_dim, exogenous_dim = get_input_dims(X_train, exogenous_data_train)

print(input_dim, exogenous_dim)

model = get_model(model=args.model_name,
                  input_dim=input_dim,
                  out_dim=y_train.shape[1],
                  lags=args.num_lags,
                  exogenous_dim=exogenous_dim,
                  seed=args.seed)

9 0


In [28]:
model

LSTM(
  (lstm): LSTM(9, 128, batch_first=True)
  (MLP_layers): Sequential(
    (0): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [29]:
def fit(model, X_train, y_train, X_val, y_val, 
        exogenous_data_train=None, exogenous_data_val=None, 
        idxs=[], # the indices of our targets in X
        log_per=1):
    
    # get exogenous data (if any)
    if exogenous_data_train is not None and len(exogenous_data_train) > 1:
        exogenous_data_train = exogenous_data_train["all"]
        exogenous_data_val = exogenous_data_val["all"]
    elif exogenous_data_train is not None and len(exogenous_data_train) == 1:
        cid = next(iter(exogenous_data_train.keys()))
        exogenous_data_train = exogenous_data_train[cid]
        exogenous_data_val = exogenous_data_val[cid]
    else:
        exogenous_data_train = None
        exogenous_data_val = None
    num_features = len(X_train[0][0])
    
    # to torch loader
    train_loader = to_torch_dataset(X_train, y_train,
                                    num_lags=args.num_lags,
                                    num_features=num_features,
                                    exogenous_data=exogenous_data_train,
                                    indices=idxs,
                                    batch_size=args.batch_size, 
                                    shuffle=False)
    val_loader = to_torch_dataset(X_val, y_val, 
                                  num_lags=args.num_lags,
                                  num_features=num_features,
                                  exogenous_data=exogenous_data_val,
                                  indices=idxs,
                                  batch_size=args.batch_size,
                                  shuffle=False)
    
    # train the model
    model = train(model, 
                  train_loader, val_loader,
                  epochs=args.epochs,
                  optimizer=args.optimizer, lr=args.lr,
                  criterion=args.criterion,
                  early_stopping=args.early_stopping,
                  patience=args.patience,
                  plot_history=args.plot_history, 
                  device=device, log_per=log_per,
                  use_carbontracker=False)
    
    return model

In [ ]:
trained_model = fit(model, X_train, y_train, X_val, y_val)